In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM,Embedding,Flatten
from keras.utils.np_utils import to_categorical
import re
from keras import backend as K 
from sklearn.metrics import confusion_matrix,classification_report

In [2]:
data = pd.read_csv('/Users/ks/Desktop/newfolder/Sentiment.csv')

In [3]:
data

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [5]:
data['sentiment'].unique()

array(['Neutral', 'Positive', 'Negative'], dtype=object)

In [6]:
data['sentiment'].value_counts()

Negative    8493
Neutral     3142
Positive    2236
Name: sentiment, dtype: int64

In [7]:
def preprocess(a):
    if a=='RT' or a.startswith('@') or a.startswith('http'):
        pass
    else:
        return a

In [8]:
data['processed_text']=data['text'].apply(lambda x: ' '.join([re.sub('[^a-zA-z0-9\s]','',i) for i in x.split(' ') if preprocess(i)!= None]).lower())

In [9]:
data_for_model=data[['processed_text','sentiment']]

In [11]:
t=Tokenizer()
t.fit_on_texts(data_for_model['processed_text'])
X=t.texts_to_sequences(data_for_model['processed_text'])
X=pad_sequences(X,maxlen=160)

In [12]:
y=pd.get_dummies(data_for_model['sentiment']).values

In [13]:
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.3,random_state=1)

In [14]:
K.clear_session()
model=Sequential()
model.add(Embedding(len(t.word_index)+1,50,input_length=160))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 160, 50)           651350    
                                                                 
 lstm (LSTM)                 (None, 128)               91648     
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
Total params: 743,385
Trainable params: 743,385
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
model.fit(train_X,train_y,epochs=10,validation_split=0.2)

Epoch 1/10
243/243 [==============================] - 125s 484ms/step - loss: 0.8550 - accuracy: 0.6319 - val_loss: 0.7976 - val_accuracy: 0.6498
Epoch 2/10
243/243 [==============================] - 113s 463ms/step - loss: 0.6599 - accuracy: 0.7220 - val_loss: 0.7802 - val_accuracy: 0.6545
Epoch 3/10
243/243 [==============================] - 112s 462ms/step - loss: 0.5189 - accuracy: 0.7883 - val_loss: 0.8588 - val_accuracy: 0.6483
Epoch 4/10
243/243 [==============================] - 111s 457ms/step - loss: 0.3956 - accuracy: 0.8431 - val_loss: 0.8749 - val_accuracy: 0.6483
Epoch 5/10
243/243 [==============================] - 112s 460ms/step - loss: 0.3146 - accuracy: 0.8764 - val_loss: 0.9568 - val_accuracy: 0.6535
Epoch 6/10
243/243 [==============================] - 112s 460ms/step - loss: 0.2754 - accuracy: 0.8907 - val_loss: 1.0150 - val_accuracy: 0.6632
Epoch 7/10
243/243 [==============================] - 112s 462ms/step - loss: 0.2428 - accuracy: 0.9028 - val_loss: 1.0783 -

In [17]:
pred=model.predict(test_X.reshape(-1,test_X.shape[1]))

In [18]:
pred_y=np.argmax(pred,axis=1)

In [19]:
act_y=np.argmax(test_y,axis=1)

In [20]:
confusion_matrix(pred_y,act_y)

array([[1891,  382,  217],
       [ 436,  431,  131],
       [ 223,  127,  324]], dtype=int64)

In [21]:
print(classification_report(pred_y,act_y))

              precision    recall  f1-score   support

           0       0.74      0.76      0.75      2490
           1       0.46      0.43      0.44       998
           2       0.48      0.48      0.48       674

    accuracy                           0.64      4162
   macro avg       0.56      0.56      0.56      4162
weighted avg       0.63      0.64      0.63      4162

